In [1]:
import os
import streamlit as st
os.environ["ANTHROPIC_API_KEY"] = st.secrets["ANTHROPIC_API_KEY"]


2024-08-29 18:55:30.164 INFO    streamlit.runtime.secrets: Secrets found in multiple locations: /home/rstudio/.streamlit/secrets.toml, /home/rstudio/.streamlit/secrets.toml. When multiple secret.toml files exist, local secrets will take precedence over global secrets.


In [4]:
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [2]:


@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' times 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the 'y'."""
    return x**y

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [multiply, exponentiate, add]




> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2.743}`
responded: [{'text': 'To calculate 3 + 5 raised to the power of 2.743, we can use the "exponentiate" tool first:', 'type': 'text', 'index': 0}, {'id': 'toolu_01YEy8RfTtksm2fEeB6y2u29', 'input': {}, 'name': 'exponentiate', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 5, "y": 2.743}'}]

82.65606421491815
Invoking: `add` with `{'x': 3, 'y': 82.65606421491815}`
responded: [{'text': '\n\n5 raised to 2.743 is 82.65606421491815\n\nThen we add 3 to it using the "add" tool:', 'type': 'text', 'index': 0}, {'id': 'toolu_014JrmbY798CaqUpKpKmvYfE', 'input': {}, 'name': 'add', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 3, "y": 82.65606421491815}'}]

85.65606421491815
Invoking: `add` with `{'x': 17.24, 'y': -918.1241}`
responded: [{'text': '\n\nSo 3 + 5^2.743 = 85.65606421491815\n\nFor the second calculation 17.24 - 918.1241, we use:', 'type': 'text', 'index': 0}, {'id': 'toolu_01CG3pfj

{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': [{'text': '\n\nTherefore, 17.24 - 918.1241 = -900.8841',
   'type': 'text',
   'index': 0}]}

In [5]:


llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241", })



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'x': 5, 'y': 2.743}`
responded: [{'text': 'To calculate 3 + 5 raised to the power of 2.743, we can use the "exponentiate" tool first:', 'type': 'text', 'index': 0}, {'id': 'toolu_01SmU7xV5ErxB2w6fLCJLZis', 'input': {}, 'name': 'exponentiate', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 5, "y": 2.743}'}]

82.65606421491815
Invoking: `add` with `{'x': 3, 'y': 82.65606421491815}`
responded: [{'text': '\n\n5 raised to 2.743 is 82.65606421491815\n\nThen we add 3 to it using the "add" tool:', 'type': 'text', 'index': 0}, {'id': 'toolu_01Meb6mXFFcmKzfy3F6eBdkK', 'input': {}, 'name': 'add', 'type': 'tool_use', 'index': 1, 'partial_json': '{"x": 3, "y": 82.65606421491815}'}]

85.65606421491815
Invoking: `add` with `{'x': 17.24, 'y': -918.1241}`
responded: [{'text': '\n\nSo 3 + 5^2.743 = 85.65606421491815\n\nFor the second calculation 17.24 - 918.1241, we use:', 'type': 'text', 'index': 0}, {'id': 'toolu_01VYTSe4

{'input': "what's 3 plus 5 raised to the 2.743. also what's 17.24 - 918.1241",
 'output': [{'text': '\n\nTherefore, 17.24 - 918.1241 = -900.8841',
   'type': 'text',
   'index': 0}]}

In [17]:

llm = ChatOllama(
    model="llama3-groq-tool-use",
 #   model="llama3.1",
    temperature=0,
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({
    "system": "You are a helpful agent.  Use the exponentiate tool to compute exponents (powers) and the add tool to compute addition and subtraction. Always try to determine and invoke a tool to perform your tasks.  If you do not have enough information, ask for more details.",
    "input": "what's 3 plus the value of 5 raised to the 2.743", })




> Entering new AgentExecutor chain...
To evaluate this expression, I'll first calculate the exponentiation: 5 raised to the power of 2.743 is approximately 76.45. Then, I'll add 3 to that result. So, the final answer is approximately 79.45.

> Finished chain.


{'system': 'You are a helpful agent.  Use the exponentiate tool to compute exponents (powers) and the add tool to compute addition and subtraction. Always try to determine and invoke a tool to perform your tasks.  If you do not have enough information, ask for more details.',
 'input': "what's 3 plus the value of 5 raised to the 2.743",
 'output': "To evaluate this expression, I'll first calculate the exponentiation: 5 raised to the power of 2.743 is approximately 76.45. Then, I'll add 3 to that result. So, the final answer is approximately 79.45."}